In [ ]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [ ]:
api = SentinelAPI('admin', 'password', 'https://apihub.copernicus.eu/apihub')

In [ ]:
footprint = geojson_to_wkt(read_geojson('malopolska.geojson'))
products = api.query(footprint,
                     date=('NOW-5DAYS', 'NOW'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 20))

# download all results from the search
# api.download_all(products)

In [ ]:
products_df = api.to_dataframe(products)
products_df